# AIM: Visualize reported availability of volunteers on greographical map

Background:
- In crisis situation, organizations might require rapid and non-bureaucratic low-level support. This code aims at generating a geographical map with the available volunteers overlayed


Requirements:
- retrieve data via json API
    table of anonymized volunteers with postcode (PLZ, later GPS coord), weekday, type (time range)
- PLZ coordinates in .xml downloaded from http://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/data.zip (generalizeable to other countries??)



# Initialization
(install and import packages)

In [ ]:
pip install matplotlib

plotly_express [https://github.com/plotly/plotly_express/]

In [ ]:
!pip install plotly_express

separate extension jupyterlab-plotly for jupyter-lab plotly [https://plotly.com/python/getting-started/#jupyterlab-support-python-35] 

In [135]:
!pip install jupyterlab==1.2 "ipywidgets>=7.5"

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  ERROR: Could not find a version that satisfies the requirement jupyterlab==1.2 (from versions: 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.13, 0.1.1, 0.1.2, 0.2.0, 0.3.0, 0.4.0, 0.4.1, 0.5.0, 0.6.0, 0.7.0, 0.8.0, 0.9.0, 0.9.1, 0.10.0, 0.11.0, 0.11.1, 0.11.2, 0.11.3, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.14.0, 0.15.0, 0.15.1, 0.16.0, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.17.4, 0.17.5, 0.18.0.dev1, 0.18.0, 0.18.1, 0.19.0, 0.20.0rc1, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21.0rc1, 0.21.0rc2, 0.21.0rc3, 0.21.0rc4, 0.21.0rc5, 0.21.0, 0.22.0rc0, 0.22.0, 0.22.1, 0.23.0rc0, 0.23.0rc1, 0.23.0, 0.23.1, 0.23.2, 0.24.0rc0, 0.2

geopandas

In [ ]:
!pip install geopandas

## list python modules with version

In [ ]:
!pip3 freeze

# code

## read volunteer data via API .json to count table

In [132]:
import requests
import json
from pandas.io.json import json_normalize
from collections import Counter
import numpy as np

# list of available time blocks
def list_of_blocks(ld):
    return list(json_normalize(d)['key'] for d in ld)

#def list_of_counts(ld):
#    return list(Counter(json_normalize(d)['key']) for d in ld)
#    return list(pd.Series(json_normalize(json_normalize(df)['capacities'][0])['key']) for d in ld)  
#   Create a mapping of the tuples formed after converting json strings of list to a python list

def occurDict(items):
    l = {}
    for j in items.index:
        d = {}
        for i in items[j]:
            if i in d:
                d[i] = d[i]+1
            else:
                d[i] = 1
        l.append(d)
    return l

#def list_of_dicts(ld):
#    return dict([(list(d.values())[1], list(d.values())[0]) for d in ld])


url = "https://staging.medicalinformatics.ch/api/v1/helpers"

payload = "{\n\t\"hash\": \"4a9cmGzA\"\n}"
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/json'
}

indf = json.loads(requests.request("POST", url, headers=headers, data = payload).text.encode('utf8'))
#df = json_normalize(json.loads(response))
 
PLZ = json_normalize(indf)['postcode']
lat = json_normalize(indf)['latitude']
lon = json_normalize(indf)['longitude']
# eventually to extract from table
a = ["MONDAY", "TUESDAY","WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]
b = ["FORENOON", "AFTERNOON", "EVENING", "NIGHT"]
blocks = ['_'.join([x,y]) for x in a for y in b]
cou = json_normalize(indf)['capacities'].apply(list_of_blocks)


# fill a dataframe
df = pd.DataFrame({'PLZ':PLZ,
                   'lat':lat,
                   'lon':lon               
                   }, dtype=float
                 )

# data frame from list of dictionaries 
#data = [{'a': 1, 'b': 2},{'a': 5, 'b': 10, 'c': 20}]
#df = pd.DataFrame(data)


print(cou)

0     [[MONDAY_NIGHT], [TUESDAY_FORENOON], [WEDNESDA...
1     [[MONDAY_AFTERNOON], [MONDAY_EVENING], [MONDAY...
2     [[MONDAY_AFTERNOON], [MONDAY_EVENING], [TUESDA...
3     [[TUESDAY_NIGHT], [WEDNESDAY_FORENOON], [WEDNE...
4     [[MONDAY_AFTERNOON], [MONDAY_EVENING], [MONDAY...
5     [[TUESDAY_FORENOON], [WEDNESDAY_NIGHT], [THURS...
6                                  [[THURSDAY_EVENING]]
7     [[MONDAY_FORENOON], [MONDAY_AFTERNOON], [MONDA...
8     [[MONDAY_FORENOON], [MONDAY_AFTERNOON], [MONDA...
9     [[MONDAY_AFTERNOON], [TUESDAY_NIGHT], [WEDNESD...
10    [[MONDAY_NIGHT], [TUESDAY_NIGHT], [THURSDAY_EV...
11    [[MONDAY_AFTERNOON], [TUESDAY_EVENING], [WEDNE...
12    [[MONDAY_FORENOON], [TUESDAY_FORENOON], [TUESD...
13    [[MONDAY_EVENING], [WEDNESDAY_EVENING], [WEDNE...
14    [[MONDAY_FORENOON], [MONDAY_EVENING], [MONDAY_...
15                                 [[FRIDAY_AFTERNOON]]
16    [[MONDAY_FORENOON], [MONDAY_AFTERNOON], [MONDA...
17    [[MONDAY_FORENOON], [MONDAY_AFTERNOON], [M

In [133]:
print(df)


       PLZ        lat         lon
0   2282.0  57.254013  -13.846746
1     46.0  67.857323   94.643538
2   8828.0 -86.629925   15.794876
3   8892.0  47.105000    9.347000
4   7059.0  -7.279339  -15.628610
5    547.0 -61.458959   67.609540
6   1975.0  46.239000    7.303000
7   5044.0  47.293000    8.089000
8   6683.0  46.306000    8.548000
9   5387.0 -89.544934  -33.062889
10  9829.0   5.347976  -77.727888
11  2541.0  56.010134  -46.198767
12   586.0  69.891960  -50.386301
13  1505.0 -19.798765   55.141315
14  6177.0 -47.336469  -25.100884
15  4028.0  63.961867  166.102008
16  5756.0 -74.962654  -70.306928
17  7411.0  46.700000    9.454000
18  4289.0  41.713063  125.196795
19  9772.0 -36.910652   43.285697
20   331.0 -38.421659 -164.665188
21  3378.0 -70.076298  -85.202063
22   945.0 -28.229461 -142.016039
23  2908.0  47.392000    6.940000
24  8044.0  47.380000    8.573000
25   387.0  28.553265  -72.268292


In [77]:
print(list(cou[10]))

[0    MONDAY_EVENING
Name: key, dtype: object, 0    WEDNESDAY_FORENOON
Name: key, dtype: object, 0    WEDNESDAY_AFTERNOON
Name: key, dtype: object, 0    THURSDAY_FORENOON
Name: key, dtype: object, 0    THURSDAY_AFTERNOON
Name: key, dtype: object, 0    FRIDAY_FORENOON
Name: key, dtype: object, 0    SATURDAY_NIGHT
Name: key, dtype: object, 0    SUNDAY_NIGHT
Name: key, dtype: object]


## generate map

In [134]:
import pandas as pd
import plotly.express as px
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")


#fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="City", hover_data=["State", "Population"],
#                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name="PLZ",
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [128]:
print(us_cities["lat"])

0      48.051764
1      33.782519
2      41.499320
3      42.262593
4      34.000710
5      41.558153
6      44.804132
7      42.473369
8      30.224090
9      43.618710
10     41.681993
11     33.867514
12     33.952602
13     39.518600
14     26.715342
15     42.519540
16     41.760585
17     39.115662
18     25.819542
19     33.751342
20     33.577863
21     38.982228
22     32.298757
23     35.262082
24     29.210815
25     39.099727
26     33.744461
27     44.767742
28     32.902902
29     33.394266
         ...    
970    29.187199
971    34.023243
972    42.011141
973    43.208137
974    47.037874
975    33.660297
976    27.800583
977    42.776201
978    41.999472
979    42.373616
980    33.538652
981    33.802516
982    33.018504
983    41.523644
984    26.368306
985    41.661210
986    42.497258
987    41.520052
988    45.500136
989    26.140363
990    36.304774
991    37.354108
992    33.195870
993    33.646966
994    42.480590
995    40.787879
996    42.408430
997    40.6220

In [130]:
print(df["latitude"])

0     57.254013
1     67.857323
2    -86.629925
3     47.105000
4     -7.279339
5    -61.458959
6     46.239000
7     47.293000
8     46.306000
9    -89.544934
10     5.347976
11    56.010134
12    69.891960
13   -19.798765
14   -47.336469
15    63.961867
16   -74.962654
17    46.700000
18    41.713063
19   -36.910652
20   -38.421659
21   -70.076298
22   -28.229461
23    47.392000
24    47.380000
25    28.553265
Name: latitude, dtype: float64


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # make the map global rather than have it zoom in to
    # the extents of any plotted data

ax.set_extent((60, 150, 55, -25))

ax.stock_img()
ax.coastlines()

ax.tissot(facecolor='purple', alpha=0.8)

plt.show()